In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from local_plot import *
from utils import *
import re


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
def extract_broadcast_D2VINS_from_log(log_content):
    # Sample log:
    # [D2VINS] Broadcast VINS Data size 4144 with 54 poses 10 extrinsic.
    # Match using regex, to match broadcast poses and extrinsic
    pattern = re.compile(r"\[D2VINS\] Broadcast VINS Data size \d+ with (\d+) poses (\d+) extrinsic.")
    matched = pattern.findall(log_content)
    matched = np.array(matched, dtype=float)
    return np.sum(matched[:, 0]) + np.sum(matched[:, 1])

def extract_broadcast_D2PGO_from_log(log_content):
    # [Drone 2] DPGO broadcast poses 249
    # Match using regex, to match broadcast poses
    pattern = re.compile(r"\[Drone \d+\] DPGO broadcast poses (\d+)")
    matched = pattern.findall(log_content)
    matched = np.array(matched, dtype=float)
    return np.sum(matched)

def extract_broadcast_poses(log_vo):
    with open(log_vo, "r") as f:
        log_content = f.read()
        broadcast_D2VINS = extract_broadcast_D2VINS_from_log(log_content)
        broadcast_D2PGO = extract_broadcast_D2PGO_from_log(log_content)
        print("Broadcast D2PGO", broadcast_D2PGO/1024/1024)
        return broadcast_D2VINS + broadcast_D2PGO*2
    
def process_bd_for_backend(path):
    broadcastsize = 0
    for i in range(1, 6):
        log_vo = f"{path}{i}/d2slam.log"
        broadcastsize += extract_broadcast_poses(log_vo)
    print("Total broadcast size", broadcastsize/1024/1024)


In [11]:
path = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2slam/swarm"
process_bd_for_backend(path)


Broadcast D2VINS 99654.0
Broadcast D2VINS 90585.0
Broadcast D2VINS 65832.0
Broadcast D2VINS 80770.0
Broadcast D2VINS 116539.0
Total broadcast size 1.6305656433105469


In [16]:
path = "/home/xuhao/data/d2slam/quadcam_7inch_n3_2023_1_14/outputs/d2slam-5-yaw/swarm"
process_bd_for_backend(path)

Broadcast D2PGO 13.668015480041504
Broadcast D2PGO 14.431467056274414
Broadcast D2PGO 14.196657180786133
Broadcast D2PGO 13.575677871704102
Broadcast D2PGO 13.843701362609863
Total broadcast size 142.60583019256592
